In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader, JSONLoader
from utils.graph_utils import read_json_file
import json

DOCS_PATH= "/home/chryssida/DATA_TUC-KRITI/SEA IMPORT/234107"
json_loader = DirectoryLoader(DOCS_PATH, glob="*unique.json", loader_cls=lambda path: JSONLoader(path, jq_schema=".[]", text_content=False), show_progress=True)
json_docs = json_loader.load()
print(json_docs)

list = read_json_file(DOCS_PATH+"/234107_unique.json")
list_of_texts = [json.dumps(j) for j in list]
print(list_of_texts)
print("\n\n")
print(list)

100%|██████████| 1/1 [00:00<00:00, 134.00it/s]

[Document(metadata={'source': '/home/chryssida/DATA_TUC-KRITI/SEA IMPORT/234107/234107_unique.json', 'seq_num': 1}, page_content='{"from": "Sofia Stafylaraki <sofia@arianmaritime.gr>", "sent": "Wednesday, December 13, 2023 4:28 AM", "to": "\\u5218\\u4e1a <lynn@globalsailing.com.cn>", "cc": "Athina Begka <operations01@arianmaritime.gr>", "subject": "1x20dv QINGDAO / THESSALONIKI S/ROSING C/ZAK", "body": "Dear Lynn,\\nProceed with this.\\nAthina \\nPlease add ref nr.\\nThanks \\nBest regards \\nSofia Stafilaraki (Mrs.)"}'), Document(metadata={'source': '/home/chryssida/DATA_TUC-KRITI/SEA IMPORT/234107/234107_unique.json', 'seq_num': 2}, page_content='{"from": "Athina Begka", "sent": "Wednesday, December 13, 2023 02:32 PM", "to": "\\u5218\\u4e1a lynn@globalsailing.com.cn", "cc": "Sofia Stafylaraki sofia@arianmaritime.gr", "subject": "234107  1x20dv QINGDAO / THESSALONIKI S/ROSING C/ZAK", "body": "Dear All, \\nGood day,\\nAdding Arian ref 234107 \\nHoliday note: our offices will be closed 

In [9]:
import asyncio
from lightrag.lightrag import LightRAG, QueryParam
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.kg.shared_storage import initialize_share_data, initialize_pipeline_status
from lightrag.utils import EmbeddingFunc
from functools import partial
from dotenv import load_dotenv

load_dotenv()

rag = LightRAG(
        working_dir="./rag_storage",
        graph_storage="Neo4JStorage",
        llm_model_func=ollama_model_complete,
        llm_model_name="llama3.1:8b",#"qwen2.5:14b",
        llm_model_max_async=4,
        llm_model_kwargs={"host": "http://localhost:11434", "options": {"num_ctx": 32768}},
        #min_rerank_score=0.5,
        vector_storage="FaissVectorDBStorage",
        #rerank_model_func=rerank_model_func,
        embedding_func=EmbeddingFunc(
            embedding_dim=768,
            max_token_size=8192,
            func=lambda texts: ollama_embed(
                texts, embed_model="nomic-embed-text", host="http://localhost:11434"
                )
        ),
        enable_llm_cache=False
    )

# Initialize database connections
await rag.initialize_storages()
# Ensure shared dicts exist
initialize_share_data()
# Initialize pipeline status for document processing
await initialize_pipeline_status()

response = await rag.aquery("Who does Sofia Stafylaraki work for?", param=QueryParam(mode="mix", top_k=7, enable_rerank=False))
print(response)


INFO: [_] Faiss index loaded with 148 vectors from ./rag_storage/faiss_index_entities.index
INFO: [_] Faiss index loaded with 178 vectors from ./rag_storage/faiss_index_relationships.index
INFO: [_] Faiss index loaded with 40 vectors from ./rag_storage/faiss_index_chunks.index
INFO: [base] Connected to neo4j at neo4j://localhost:7687
INFO: [base] Ensured B-Tree index on entity_id for base in neo4j
INFO: [base] Found existing index 'entity_id_fulltext_idx' with state: ONLINE
INFO: [base] Full-text index 'entity_id_fulltext_idx' already exists and is online. Skipping recreation.
INFO: LLM func: 4 new workers initialized (Timeouts: Func: 180s, Worker: 360s, Health Check: 375s)
INFO: Query nodes: Sofia Stafylaraki (top_k:7, cosine:0.2)
INFO: Embedding func: 8 new workers initialized (Timeouts: Func: 30s, Worker: 60s, Health Check: 75s)
INFO: Local query: 7 entites, 34 relations
INFO: Query edges: Employment, Professional affiliations (top_k:7, cosine:0.2)
INFO: Global query: 11 entites, 7 

### Role of Sofia Stafylaraki
According to the provided context, Sofia Stafylaraki works for Arian Maritime. This is supported by multiple email exchanges that mention her as a representative or employee of the company.

### Specific Details about Arian Maritime
Arian Maritime appears to be a shipping company that manages various aspects of shipping and logistics operations. The organization has employees, including Sofia Stafylaraki and Athina Begka, who are involved in operational tasks and communication with clients or partners.

### Evidence from Email Communications
The email exchanges provided in the context demonstrate Sofia's involvement in managing shipping operations for Arian Maritime. Her emails often reference shipping schedules, cargo details, and updates on vessel arrivals and departures.

### Conclusion about Sofia Stafylaraki's Role
Based on the available information, it is clear that Sofia Stafylaraki plays a significant role at Arian Maritime, contributing to the com